In [49]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sys
import math

In [2]:
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.1 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s %s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [3]:
meta = pd.read_csv('../data/combined.tsv', sep='\t', header=0, index_col=None)

In [4]:
meta.head()

,Virus name,Accession ID,Collection date,Location,Host,Additional location information,Gender,Patient age,Patient status,Passage,Specimen,Additional host information,Lineage,Clade
0,hCoV-19/Mayotte/IPP00846/2021,EPI_ISL_1013099,2021-01-11,Africa / Mayotte / Mamoudzou,Human,NaN,Female,61,Inpatient,Original,NaN,NaN,B.1.160,GH
1,hCoV-19/Mayotte/IPP03105/2021,EPI_ISL_1013300,2021-01-28,Africa / Mayotte / Mamoudzou,Human,NaN,Female,8,Inpatient,Original,Nasopharyngeal,NaN,B.1.351,GH
2,hCoV-19/Mayotte/IPP03108/2021,EPI_ISL_1013301,2021-01-28,Africa / Mayotte / Mamoudzou,Human,NaN,Female,49,Inpatient,Original,Nasopharyngeal,NaN,B.1.351,GH
3,hCoV-19/Mayotte/IPP03109/2021,EPI_ISL_1013302,2021-01-28,Africa / Mayotte / Mamoudzou,Human,NaN,Male,78,Inpatient,Original,Nasopharyngeal,NaN,B.1.351,GH
4,hCoV-19/Mayotte/IPP03110/2021,EPI_ISL_1013303,2021-01-28,Africa / Mayotte / Mamoudzou,Human,NaN,Male,34,Inpatient,Original,Nasopharyngeal,NaN,B.1.351,GH


In [5]:
meta['Patient status'].unique()

array(['Inpatient', 'Hospitalized', 'Live', 'Isolation',
       'Not hospitalized', 'Released', 'Recovered', 'Post mortem',
       'Outpatient', 'inpatient', 'uknown', 'Stable in quarantine',
       'Deceased', 'outpatient', 'Hospitalised', 'Patient status',
       'Discharged', 'Hospitalized (Stable)', 'Symptomatic',
       'Asymptomatic', 'Suspected Corona',
       'Suspected coronavirus infection', 'ICU', 'Overseas inflow',
       'Released, Live', 'Hospitalized (Released)',
       'Hospitalized, Asymptomatic, Alive',
       'Hospitalized, Discharged, Alive', 'Hospitalized / ICU', 'Mild',
       'Fever', 'Non-hospitalized, symptomps: sore throat',
       'Pneumonia (chest X-ray)', 'Intensive Care Unit',
       'Hospitalized (Deceased)', 'Discharged after recovery',
       'No clinical signs', 'Quarantine', 'Facility quarantine',
       'Domestic infection',
       'Initially hospitalized, but now improved and discharged',
       'Hospitalized (Mild)', 'Asymptomatic/Released', 'DAMA'

In [6]:
high_risk = {
    'hospitalized': meta[(meta['Patient status'].str.lower().str.contains('hospitalized|hospitalizes|hospitlalized', na=False))
                         & (~meta['Patient status'].str.lower().str.contains('non|not', na=False))],
    'inpatient': meta[meta['Patient status'].str.lower().str.contains('inpatient', na=False)],
    'deceased': meta[meta['Patient status'].str.lower().str.contains('deceased', na=False)],
    'severe': meta[meta['Patient status'].str.lower().str.contains('severe', na=False)]
}

low_risk = {
    'outpatient': meta[meta['Patient status'].str.lower().str.contains('outpatient', na=False)],
    'asymptomatic': meta[meta['Patient status'].str.lower().str.contains('asymptomatic', na=False)],
    'mild': meta[meta['Patient status'].str.lower().str.contains('mild', na=False)],
    'home': meta[meta['Patient status'].str.lower().str.contains('home|not hospitalized', na=False)]
}

In [7]:
high_risk_df = pd.concat(high_risk.values(), ignore_index=True).drop_duplicates(ignore_index=True)
low_risk_df = pd.concat(low_risk.values(), ignore_index=True).drop_duplicates(ignore_index=True)

In [8]:
high_risk_df['severe'] = 1
low_risk_df['severe'] = 0

In [9]:
final_meta_df = pd.concat([high_risk_df, low_risk_df])[['Virus name', 'Accession ID', 'Collection date', 'severe']]

In [10]:
final_meta_df['name'] = final_meta_df['Virus name'] + '|' + final_meta_df['Accession ID'] + '|' + final_meta_df['Collection date']

In [11]:
final_meta_df.head()

,Virus name,Accession ID,Collection date,severe,name
0,hCoV-19/Nigeria/Lagos01/2020,EPI_ISL_413550,2020-02-27,1,hCoV-19/Nigeria/Lagos01/2020|EPI_ISL_413550|20...
1,hCoV-19/DRC/2363/2020,EPI_ISL_437194,2020-04-14,1,hCoV-19/DRC/2363/2020|EPI_ISL_437194|2020-04-14
2,hCoV-19/Nigeria/KW298-CV48/2020,EPI_ISL_487107,2020-05-08,1,hCoV-19/Nigeria/KW298-CV48/2020|EPI_ISL_487107...
3,hCoV-19/Ghana/UHAS-K674/2020,EPI_ISL_628760,2020-07-07,1,hCoV-19/Ghana/UHAS-K674/2020|EPI_ISL_628760|20...
4,hCoV-19/Ghana/UHAS-H009/2020,EPI_ISL_628761,2020-06-16,1,hCoV-19/Ghana/UHAS-H009/2020|EPI_ISL_628761|20...


In [113]:
vcf = pd.read_csv('../data/snpeff_fixed_updated.vcf', sep='\t', skiprows=8, low_memory=False, memory_map=True)

In [114]:
vcf.shape

(15176, 26858)

In [115]:
vcf.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,"NC_045512.2Severeacuterespiratorysyndromecoronavirus2isolateWuhan-Hu-1,completegenome",...,hCoV-19/Colombia/AMA-INS-VG-632/2021|EPI_ISL_956288|2021-01-13,hCoV-19/Colombia/AMA-INS-VG-633/2021|EPI_ISL_956289|2021-01-12,hCoV-19/Colombia/AMA-INS-VG-634/2021|EPI_ISL_956290|2021-01-12,hCoV-19/Colombia/AMA-INS-VG-635/2021|EPI_ISL_956291|2021-01-22,hCoV-19/Colombia/AMA-INS-VG-636/2021|EPI_ISL_956292|2021-01-22,hCoV-19/Colombia/AMA-INS-VG-637/2021|EPI_ISL_956293|2021-01-20,hCoV-19/Colombia/AMA-INS-VG-638/2021|EPI_ISL_956294|2021-01-18,hCoV-19/Colombia/AMA-INS-VG-367/2021|EPI_ISL_956297|2021-01-04,hCoV-19/Colombia/VAC-INS-RI-011/2021|EPI_ISL_956298|2021-01-13,hCoV-19/Brazil/RS-8680/2021|EPI_ISL_983865|2021-02-01
0,NC_045512.2,1,.,A,"*,C,G,T,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
1,NC_045512.2,2,.,T,"*,C,A,G,K,Y,W",.,.,ANN=A|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
2,NC_045512.2,3,.,T,"*,C,A,W,G,K",.,.,ANN=A|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
3,NC_045512.2,4,.,A,"*,G,T,W,C,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
4,NC_045512.2,5,.,A,"*,T,G,C,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1


In [116]:
ids = set(final_meta_df['Accession ID'])

In [117]:
len(ids)

8921

In [118]:
index0 = vcf.columns.str.split('|').str[0]
index1 = vcf.columns.str.split('|').str[1]

In [119]:
index0[:10]

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT',
       'NC_045512.2Severeacuterespiratorysyndromecoronavirus2isolateWuhan-Hu-1,completegenome'],
      dtype='object')

In [120]:
new_cols = list(index0[:10]) + list(index1[10:])

In [121]:
vcf.columns = new_cols

In [122]:
vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,"NC_045512.2Severeacuterespiratorysyndromecoronavirus2isolateWuhan-Hu-1,completegenome",...,EPI_ISL_956288,EPI_ISL_956289,EPI_ISL_956290,EPI_ISL_956291,EPI_ISL_956292,EPI_ISL_956293,EPI_ISL_956294,EPI_ISL_956297,EPI_ISL_956298,EPI_ISL_983865
0,NC_045512.2,1,.,A,"*,C,G,T,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
1,NC_045512.2,2,.,T,"*,C,A,G,K,Y,W",.,.,ANN=A|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
2,NC_045512.2,3,.,T,"*,C,A,W,G,K",.,.,ANN=A|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
3,NC_045512.2,4,.,A,"*,G,T,W,C,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
4,NC_045512.2,5,.,A,"*,T,G,C,M",.,.,ANN=C|upstream_gene_variant|MODIFIER|ORF1ab|GU...,GT,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,NC_045512.2,29899,.,A,"*,C,T,G,H,M",.,.,ANN=C|downstream_gene_variant|MODIFIER|S|GU280...,GT,0,...,1,1,1,1,1,1,1,1,1,1
15172,NC_045512.2,29900,.,A,"*,T,G,C,M,W",.,.,ANN=C|downstream_gene_variant|MODIFIER|S|GU280...,GT,0,...,1,1,1,1,1,1,1,1,1,1
15173,NC_045512.2,29901,.,A,"*,T,G,C",.,.,ANN=C|downstream_gene_variant|MODIFIER|S|GU280...,GT,0,...,1,1,1,1,1,1,1,1,1,1
15174,NC_045512.2,29902,.,A,"*,T,C,G",.,.,ANN=C|downstream_gene_variant|MODIFIER|S|GU280...,GT,0,...,1,1,1,1,1,1,1,1,1,1


In [154]:
vcf_subset = vcf.loc[:, vcf.columns.isin(ids)]

In [155]:
vcf_subset

,EPI_ISL_455412,EPI_ISL_455413,EPI_ISL_455415,EPI_ISL_455418,EPI_ISL_455419,EPI_ISL_455422,EPI_ISL_455423,EPI_ISL_455424,EPI_ISL_455426,EPI_ISL_455427,...,EPI_ISL_942930,EPI_ISL_942931,EPI_ISL_954090,EPI_ISL_954094,EPI_ISL_954095,EPI_ISL_954096,EPI_ISL_954097,EPI_ISL_954098,EPI_ISL_954099,EPI_ISL_983865
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15172,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15173,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15174,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [153]:
severe_acc = set(final_meta_df.loc[final_meta_df['severe'] == 1]['Accession ID'])
mild_acc = set(final_meta_df.loc[final_meta_df['severe'] == 0]['Accession ID'])

In [201]:
var_id = []
severe_tot = []
mild_tot = []
frequency = []
var_type = []

In [202]:
for i,row in vcf_subset.iterrows():
    progress(i, len(vcf_subset))
    
    for num,var in enumerate(vcf.iloc[i]['ALT']):
        severe_num = 0
        mild_num = 0
        
        all_anns = [v.split('|')[0] for v in vcf.iloc[i]['INFO'].split(',')]
        all_anns[0] = all_anns[0].replace('ANN=', '')
          
        if var in ['A', 'T', 'C', 'G']:
            index = all_anns.index(var)
            var_id.append('{0}{1}>{2}'.format(vcf.iloc[i]['POS'], vcf.iloc[i]['REF'], var))
            var_type.append(vcf.iloc[i]['INFO'].split(',')[index].split('|')[1])
            
            frequency.append(len(vcf_subset.columns[(vcf_subset == num + 1).iloc[i]]) / len(vcf_subset.iloc[i]))
            
            for col in vcf_subset.columns[(vcf_subset == num + 1).iloc[i]]:
                if col in severe_acc:
                    severe_num += 1
                else:
                    mild_num += 1
                    
            severe_tot.append(severe_num)
            mild_tot.append(mild_num)
            
        else:
            continue

In [203]:
final_df = pd.DataFrame({'variant': var_id, 'num_severe': severe_tot, 'num_mild': mild_tot, 'var_type': var_type, 
                         'frequency': frequency})

In [204]:
final_df

,variant,num_severe,num_mild,var_type,frequency
0,1A>C,2,0,upstream_gene_variant,0.000224
1,1A>G,0,0,upstream_gene_variant,0.000000
2,1A>T,0,0,upstream_gene_variant,0.000000
3,2T>C,3,1,upstream_gene_variant,0.000449
4,2T>A,0,0,upstream_gene_variant,0.000000
...,...,...,...,...,...
17909,29902A>C,0,0,downstream_gene_variant,0.000000
17910,29902A>G,0,0,downstream_gene_variant,0.000000
17911,29903A>T,0,0,downstream_gene_variant,0.000000
17912,29903A>C,0,0,downstream_gene_variant,0.000000


In [208]:
final_df = final_df.loc[final_df['frequency'] != 0]

In [209]:
final_df.to_csv('../data/counts.tsv', sep='\t', index=False, header=True)